## Agent ReAct

In [1]:
!pip install -q langchain langchain-openai python-dotenv


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Model gotowy.")


Model gotowy.


### Definiowanie narzędzi

In [3]:
from langchain.tools import tool

@tool
def multiply_numbers(x: int, y: int) -> int:
    """Mnoży dwie liczby całkowite."""
    return x * y

@tool
def company_info(name: str) -> str:
    """Zwraca podstawowe informacje o firmie."""
    data = {
        "OpenAI": "Firma zajmująca się sztuczną inteligencją, twórca ChatGPT.",
        "LangChain": "Framework do budowania aplikacji AI z LLM.",
        "Tesla": "Producent samochodów elektrycznych."
    }
    return data.get(name, "Nie mam informacji o tej firmie.")


### Agent ReAct

In [5]:
from langchain.agents import initialize_agent, AgentType

tools = [multiply_numbers, company_info]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # ReAct Agent
    verbose=True  # pokazuje reasoning krok po kroku
)


### Test agenta

In [6]:
# Zadanie matematyczne
print("=== Zadanie 1 ===")
agent.run("Policz 23 * 7")

# Zadanie informacyjne
print("\n=== Zadanie 2 ===")
agent.run("Powiedz mi, czym zajmuje się firma LangChain.")

=== Zadanie 1 ===


> Entering new AgentExecutor chain...


/tmp/ipykernel_7598/2976062710.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Policz 23 * 7")


Thought: I need to multiply the numbers 23 and 7. 
Action:
```
{
  "action": "multiply_numbers",
  "action_input": {
    "x": 23,
    "y": 7
  }
}
```

Observation: 161
Thought:I have calculated the multiplication of 23 and 7, which equals 161. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "23 * 7 = 161"
}
```


> Finished chain.

=== Zadanie 2 ===


> Entering new AgentExecutor chain...
Thought: Chcę uzyskać podstawowe informacje o firmie LangChain, aby odpowiedzieć na pytanie. 
Action:
```
{
  "action": "company_info",
  "action_input": {
    "name": "LangChain"
  }
}
```

Observation: Framework do budowania aplikacji AI z LLM.
Thought:Myślę, że mam wystarczające informacje, aby odpowiedzieć na pytanie. LangChain to framework do budowania aplikacji AI z wykorzystaniem modeli językowych (LLM). 

Action:
```
{
  "action": "Final Answer",
  "action_input": "LangChain to framework do budowania aplikacji AI z wykorzystaniem modeli językowych (LLM)."
}
```


> Finished chai

'LangChain to framework do budowania aplikacji AI z wykorzystaniem modeli językowych (LLM).'

### Dodanie nowego narzędzia (API z kursem walut)

In [8]:
import requests

@tool
def get_exchange_rate(currency: str) -> str:
    """Zwraca kurs wymiany waluty względem USD."""
    url = f"https://api.exchangerate.host/latest?base=USD&symbols={currency.upper()}"
    response = requests.get(url).json()
    rate = response["rates"].get(currency.upper())
    return f"1 USD = {rate} {currency.upper()}"

tools = [multiply_numbers, company_info, get_exchange_rate]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

print("\n=== Zadanie 3 ===")
agent.run("Jaki jest obecnie kurs euro względem dolara?")



=== Zadanie 3 ===


> Entering new AgentExecutor chain...
Thought: Aby uzyskać aktualny kurs euro względem dolara, muszę użyć odpowiedniego narzędzia do sprawdzenia kursu wymiany walut. 
Action:
```
{
  "action": "get_exchange_rate",
  "action_input": {
    "currency": "EUR"
  }
}
```


KeyError: 'rates'